In [1]:
import pandas as pd
import scipy.stats as stats
import researchpy as rp
import statsmodels.api as sm
from statsmodels.formula.api import ols
import os
import ast
import numpy as np

import matplotlib.pyplot as plt


# Analysis <font color=blue>after </font> combining RF, DT and SVM into 1C 10 min

In [2]:
def get_class (model):
    if isinstance(model, str):
        model = '{' +'}'.join('{'.join(model.split('{')[1:]).split('}')[0:1]) + '}'
        m = ast.literal_eval(model)
        if 'classifier:__choice__' in m:
            classifier = m['classifier:__choice__']
        else:
            classifier = ''
        return classifier
    else:
        return ''


def parse_tpot(directory):
    result = pd.DataFrame(columns=['dataset', 'accuracy', 'model', 'precision', 'recall', 'f1score', 'time_budget', 'methods'])
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.csv'):
                method = file.split('.')[0].split('_')[2]
                time_budget = file.split('.')[0].split('_')[3]
                run = []
                sub_result = pd.read_csv(os.path.join(subdir, file))
                sub_result.rename(columns={'Unnamed: 0': 'dataset'}, inplace = True)
                run.append(sub_result[['dataset', 'accuracy_1', 'model_1', 'precision_1', 'recall_1', 'f1score_1']])
                run.append(sub_result[['dataset', 'accuracy_2', 'model_2', 'precision_2', 'recall_2', 'f1score_2']])
                run.append(sub_result[['dataset', 'accuracy_3', 'model_3', 'precision_3', 'recall_3', 'f1score_3']])
                for i in range(3):
                    run[i].rename(columns={'accuracy_'+str(i+1): 'accuracy',
                                           'model_'+str(i+1): 'model',
                                           'precision_'+str(i+1): 'precision',
                                           'recall_'+str(i+1): 'recall',
                                           'f1score_'+str(i+1): 'f1score'}, inplace=True)
                    run[i]['methods'] = str(method)
                    run[i]['time_budget'] = int(time_budget)
                    result = pd.concat([result, run[i]], axis=0, sort=True, ignore_index=True)
    result.model = result.model.apply(get_class)
    result = result[~np.isnan(result.f1score)]
    return result
parse_tpot(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/")

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\pandas\core\frame.py:4223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

,accuracy,dataset,f1score,methods,model,precision,recall,time_budget
0,0.949153,AP_Breast_Lung,0.911765,3C,,0.911765,0.911765,10
4,0.947368,audiology,0.918919,3C,,0.944444,0.894737,10
5,0.700000,dataset_31_credit-g,0.788732,3C,,0.782123,0.795455,10
7,0.900807,gina_agnostic,0.892500,3C,,0.915385,0.870732,10
8,0.965974,hiva_agnostic,0.333333,3C,,0.692308,0.219512,10
...,...,...,...,...,...,...,...,...
891,0.597500,wine-quality-red,0.597500,SVC,,0.597500,0.597500,60
895,0.395026,eye_movements,0.395026,SVC,,0.395026,0.395026,60
897,0.645570,solar-flare_1,0.645570,SVC,,0.645570,0.645570,60
898,0.166667,umistfacescropped,0.166667,SVC,,0.166667,0.166667,60


In [3]:
#df = pd.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/atmout.xlsx")
df = parse_tpot(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/")
df = df[['dataset', 'time_budget', 'methods', 'f1score']]
# df.methods = df.methods.replace("['logreg', 'svm', 'sgd', 'dt', 'et', 'rf', 'gnb', 'mnb', 'bnb', 'gp', 'pa', 'knn', 'mlp', 'ada']", "fc")
# df.methods = df.methods.replace("['rf', 'dt', 'svm']", "3c")
# df.methods = df.methods.replace("['logreg', 'dt', 'knn']", "def")
# df.methods = df.methods.replace("['svm']", "1c")
# df.methods = df.methods.replace("['dt']", "1c")
# df.methods = df.methods.replace("['rf']", "1c")

df.methods = df.methods.replace("default", "fc")
df.methods = df.methods.replace("3C", "3c")
df.methods = df.methods.replace("SVC", "1c")
df.methods = df.methods.replace("DT", "1c")
df.methods = df.methods.replace("RF", "1c")


df1c = df[df.methods=='1c']
df3c = df[df.methods=='3c']
dffc = df[df.methods=='fc']
fsIs_b = ['vowel', 'openml_phpJNxH0q', 'dataset_31_credit-g', 'dataset_40_sonar']
fsIs_m = ['solar-flare_1', 'wine-quality-red', 'dataset_39_ecoli', 'synthetic_control']
fsIl_b = ['AirlinesCodrnaAdult', 'MagicTelescope', 'electricity-normalized', 'phpmPOD5A']
fsIl_m = ['pokerhand-normalized', 'eye_movements', 'avila-tr']
flIs_b = ['audiology', 'arrhythmia', 'AP_Breast_Lung', 'AP_Omentum_Ovary']
flIs_m = ['Amazon', 'umistfacescropped', 'phpGUrE90']
flIl_b = ['gina_agnostic', 'hiva_agnostic', 'phpZrCzJR', 'phprAeXmK']
flIl_m = ['KDDCup99', 'connect-4', 'dataset_60_waveform-5000', 'dataset_186_satimage']
df_binary = df[df.dataset.isin(fsIs_b) | df.dataset.isin(fsIl_b) | df.dataset.isin(flIs_b) | df.dataset.isin(flIl_b)]
df_multi = df[df.dataset.isin(fsIs_m) | df.dataset.isin(fsIl_m) | df.dataset.isin(flIs_m) | df.dataset.isin(flIl_m)]
#df = df_binary
df.shape

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(524, 4)

In [4]:
rp.summary_cont(df1c['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,288.0,0.632771,0.30498,0.017971,0.597399,0.668143


In [5]:
rp.summary_cont(df3c['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,103.0,0.798783,0.193445,0.019061,0.760976,0.83659


In [6]:
rp.summary_cont(dffc['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,133.0,0.801595,0.175995,0.015261,0.771407,0.831782


In [7]:
stats.f_oneway(df1c['f1score'][df1c['time_budget'] == 10], 
             df1c['f1score'][df1c['time_budget'] == 30],
             df1c['f1score'][df1c['time_budget'] == 60])

F_onewayResult(statistic=0.26434398944321374, pvalue=0.767897431742273)

In [8]:
stats.f_oneway(df3c['f1score'][df3c['time_budget'] == 10], 
             df3c['f1score'][df3c['time_budget'] == 30],
             df3c['f1score'][df3c['time_budget'] == 60])

F_onewayResult(statistic=0.19069623078059286, pvalue=0.8266833872007431)

In [9]:
stats.f_oneway(dffc['f1score'][dffc['time_budget'] == 10], 
             dffc['f1score'][dffc['time_budget'] == 30],
             dffc['f1score'][dffc['time_budget'] == 60])

F_onewayResult(statistic=0.22913659423437016, pvalue=0.7955403837904598)

In [10]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model1c = ols('f1score ~ C(time_budget)', df1c).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model1c.df_model: .0f},{model1c.df_resid: .0f}) = {model1c.fvalue: .3f}, p = {model1c.f_pvalue: .4f}")
    model1c.summary()

Overall model F( 2, 285) =  0.264, p =  0.7679


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2643
Date:                Sun, 03 Nov 2019   Prob (F-statistic):              0.768
Time:                        22:00:17   Log-Likelihood:                -65.884
No. Observations:                 288   AIC:                             137.8
Df Residuals:                     285   BIC:                             148.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.6523      0.032     20.127      0.000       0.589       0.716
C(time_budget)[T.30]    -0.0276      0.045     -0.615      0.539      -0.116       0.061
C(time_budget)[T.60]    -0.0290      0.044     -0.654      0.514      -0.116       0.058
==============================================================================
Omnibus:                       29.121   Durbin-Watson:                   2.113
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               31.172
Skew:                          -0.761   Prob(JB):                     1.70e-07
Kurtosis:                       2.470   Cond. No.                         3.86
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model3c = ols('f1score ~ C(time_budget)', df3c).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model3c.df_model: .0f},{model3c.df_resid: .0f}) = {model3c.fvalue: .3f}, p = {model3c.f_pvalue: .4f}")
    model3c.summary()

Overall model F( 2, 100) =  0.191, p =  0.8267


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.1907
Date:                Sun, 03 Nov 2019   Prob (F-statistic):              0.827
Time:                        22:00:17   Log-Likelihood:                 23.752
No. Observations:                 103   AIC:                            -41.50
Df Residuals:                     100   BIC:                            -33.60
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.8136      0.034     23.969      0.000       0.746       0.881
C(time_budget)[T.30]    -0.0135      0.049     -0.277      0.782      -0.110       0.083
C(time_budget)[T.60]    -0.0284      0.046     -0.616      0.539      -0.120       0.063
==============================================================================
Omnibus:                       15.806   Durbin-Watson:                   2.128
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               19.004
Skew:                          -1.052   Prob(JB):                     7.47e-05
Kurtosis:                       3.061   Cond. No.                         3.81
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    modelfc = ols('f1score ~ C(time_budget)', dffc).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({modelfc.df_model: .0f},{modelfc.df_resid: .0f}) = {modelfc.fvalue: .3f}, p = {modelfc.f_pvalue: .4f}")
    modelfc.summary()

Overall model F( 2, 130) =  0.229, p =  0.7955


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.2291
Date:                Sun, 03 Nov 2019   Prob (F-statistic):              0.796
Time:                        22:00:17   Log-Likelihood:                 43.078
No. Observations:                 133   AIC:                            -80.16
Df Residuals:                     130   BIC:                            -71.48
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                0.7882      0.030     26.342      0.000       0.729       0.847
C(time_budget)[T.30]     0.0097      0.040      0.243      0.808      -0.069       0.089
C(time_budget)[T.60]     0.0253      0.039      0.655      0.513      -0.051       0.102
==============================================================================
Omnibus:                       30.086   Durbin-Watson:                   2.412
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               41.965
Skew:                          -1.234   Prob(JB):                     7.72e-10
Kurtosis:                       4.217   Cond. No.                         4.16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [13]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df1c['f1score'], df1c['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30  -0.0276 0.7924 -0.1333 0.0781  False
    10     60   -0.029   0.77 -0.1335 0.0755  False
    30     60  -0.0014    0.9 -0.1036 0.1008  False
---------------------------------------------------


In [14]:
mc = MultiComparison(df3c['f1score'], df3c['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30  -0.0135    0.9 -0.1296 0.1025  False
    10     60  -0.0284 0.7922 -0.1381 0.0813  False
    30     60  -0.0149    0.9 -0.1265 0.0967  False
---------------------------------------------------


In [15]:
mc = MultiComparison(dffc['f1score'], dffc['time_budget'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    10     30   0.0097    0.9 -0.0849 0.1043  False
    10     60   0.0253   0.77 -0.0662 0.1167  False
    30     60   0.0156 0.8964 -0.0695 0.1007  False
---------------------------------------------------


# Analysis <font color=blue>without </font> combining RF, DT and SVM into 1C 10 min

In [16]:
df = pd.read_excel(r"C:\Users\HassanEldeeb\Documents\GitHub\AutoMLBenchmarking\logs_search_space/skout.xlsx")
df = df[['time_budget', 'methods', 'f1score']]
df.methods = df.methods.replace("['adaboost', 'bernoulli_nb', 'decision_tree', 'extra_trees', 'gaussian_nb', 'gradient_boosting', 'k_nearest_neighbors', 'lda', 'liblinear_svc', 'libsvm_svc', 'multinomial_nb', 'passive_aggressive', 'qda', 'random_forest', 'sgd']", "fc")
df.methods = df.methods.replace("['decision_tree', 'libsvm_svc', 'random_forest']", "3c")
df.methods = df.methods.replace("['libsvm_svc']", "svc")
df.methods = df.methods.replace("['decision_tree']", "dt")
df.methods = df.methods.replace("['random_forest']", "rf")
df10 = df[df.time_budget==10]
df30 = df[df.time_budget==30]
df60 = df[df.time_budget==60]
df.shape

(1350, 3)

In [17]:
rp.summary_cont(df10['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,450.0,0.795257,0.216084,0.010186,0.775238,0.815276


In [18]:
rp.summary_cont(df30['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,450.0,0.815889,0.182823,0.008618,0.798951,0.832826


In [19]:
rp.summary_cont(df60['f1score'])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,f1score,450.0,0.812907,0.181568,0.008559,0.796086,0.829728


In [20]:
stats.f_oneway(df10['f1score'][df10['methods'] == '1c'], 
             df10['f1score'][df10['methods'] == '3c'],
             df10['f1score'][df10['methods'] == 'fc'])

C:\Users\HassanEldeeb\Anaconda3\lib\site-packages\scipy\stats\stats.py:3225: RuntimeWarning: invalid value encountered in double_scalars
  ssbn += _square_of_sums(a - offset) / len(a)


F_onewayResult(statistic=nan, pvalue=nan)

In [21]:
stats.f_oneway(df30['f1score'][df30['methods'] == '1c'], 
             df30['f1score'][df30['methods'] == '3c'],
             df30['f1score'][df30['methods'] == 'fc'])

F_onewayResult(statistic=nan, pvalue=nan)

In [22]:
stats.f_oneway(df60['f1score'][df60['methods'] == '1c'], 
             df60['f1score'][df60['methods'] == '3c'],
             df60['f1score'][df60['methods'] == 'fc'])

F_onewayResult(statistic=nan, pvalue=nan)

In [23]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model10 = ols('f1score ~ C(methods)', df10).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model10.df_model: .0f},{model10.df_resid: .0f}) = {model10.fvalue: .3f}, p = {model10.f_pvalue: .4f}")
    model10.summary()

Overall model F( 4, 445) =  5.003, p =  0.0006


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     5.003
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           0.000594
Time:                        22:00:18   Log-Likelihood:                 61.315
No. Observations:                 450   AIC:                            -112.6
Df Residuals:                     445   BIC:                            -92.08
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8296      0.022     37.065      0.000       0.786       0.874
C(methods)[T.dt]     -0.0510      0.032     -1.612      0.108      -0.113       0.011
C(methods)[T.fc]      0.0051      0.032      0.161      0.872      -0.057       0.067
C(methods)[T.rf]     -0.0109      0.032     -0.345      0.730      -0.073       0.051
C(methods)[T.svc]    -0.1147      0.032     -3.625      0.000      -0.177      -0.053
==============================================================================
Omnibus:                      150.624   Durbin-Watson:                   0.421
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              382.223
Skew:                          -1.681   Prob(JB):                     1.00e-83
Kurtosis:                       6.014   Cond. No.                         5.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model30 = ols('f1score ~ C(methods)', df30).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model30.df_model: .0f},{model30.df_resid: .0f}) = {model30.fvalue: .3f}, p = {model30.f_pvalue: .4f}")
    model30.summary()

Overall model F( 4, 445) =  3.572, p =  0.0070


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.572
Date:                Sun, 03 Nov 2019   Prob (F-statistic):            0.00700
Time:                        22:00:18   Log-Likelihood:                 133.75
No. Observations:                 450   AIC:                            -257.5
Df Residuals:                     445   BIC:                            -236.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8412      0.019     44.147      0.000       0.804       0.879
C(methods)[T.dt]     -0.0384      0.027     -1.425      0.155      -0.091       0.015
C(methods)[T.fc]     -0.0019      0.027     -0.071      0.943      -0.055       0.051
C(methods)[T.rf]     -0.0031      0.027     -0.115      0.908      -0.056       0.050
C(methods)[T.svc]    -0.0831      0.027     -3.082      0.002      -0.136      -0.030
==============================================================================
Omnibus:                      129.208   Durbin-Watson:                   0.475
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              295.681
Skew:                          -1.479   Prob(JB):                     6.22e-65
Kurtosis:                       5.650   Cond. No.                         5.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
    # Fits the model with the interaction term
    # This will also automatically include the main effects for each factor
    model60 = ols('f1score ~ C(methods)', df60).fit()

    # Seeing if the overall model is significant
    print(f"Overall model F({model60.df_model: .0f},{model60.df_resid: .0f}) = {model60.fvalue: .3f}, p = {model60.f_pvalue: .4f}")
    model10.summary()

Overall model F( 4, 445) =  2.206, p =  0.0674


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                f1score   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     5.003
Date:                Sun, 03 Nov 2019   Prob (F-statistic):           0.000594
Time:                        22:00:18   Log-Likelihood:                 61.315
No. Observations:                 450   AIC:                            -112.6
Df Residuals:                     445   BIC:                            -92.08
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8296      0.022     37.065      0.000       0.786       0.874
C(methods)[T.dt]     -0.0510      0.032     -1.612      0.108      -0.113       0.011
C(methods)[T.fc]      0.0051      0.032      0.161      0.872      -0.057       0.067
C(methods)[T.rf]     -0.0109      0.032     -0.345      0.730      -0.073       0.051
C(methods)[T.svc]    -0.1147      0.032     -3.625      0.000      -0.177      -0.053
==============================================================================
Omnibus:                      150.624   Durbin-Watson:                   0.421
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              382.223
Skew:                          -1.681   Prob(JB):                     1.00e-83
Kurtosis:                       6.014   Cond. No.                         5.83
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison

mc = MultiComparison(df10['f1score'], df10['methods'])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     dt   -0.051 0.4908 -0.1377  0.0357  False
    3c     fc   0.0051    0.9 -0.0816  0.0918  False
    3c     rf  -0.0109    0.9 -0.0976  0.0758  False
    3c    svc  -0.1147  0.003 -0.2014  -0.028   True
    dt     fc   0.0561  0.392 -0.0306  0.1428  False
    dt     rf   0.0401 0.6869 -0.0466  0.1268  False
    dt    svc  -0.0637 0.2614 -0.1504   0.023  False
    fc     rf   -0.016    0.9 -0.1027  0.0707  False
    fc    svc  -0.1198 0.0016 -0.2065 -0.0331   True
    rf    svc  -0.1038 0.0098 -0.1905 -0.0171   True
----------------------------------------------------


In [27]:
mc = MultiComparison(df30['f1score'], df30['methods'])
mc_results = mc.tukeyhsd(model30.f_pvalue)
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.01
group1 group2 meandiff p-adj   lower  upper  reject
---------------------------------------------------
    3c     dt  -0.0384 0.5968 -0.1295 0.0527  False
    3c     fc  -0.0019    0.9  -0.093 0.0892  False
    3c     rf  -0.0031    0.9 -0.0942  0.088  False
    3c    svc  -0.0831 0.0185 -0.1742 0.0081  False
    dt     fc   0.0365 0.6372 -0.0546 0.1276  False
    dt     rf   0.0353 0.6621 -0.0558 0.1264  False
    dt    svc  -0.0446 0.4632 -0.1358 0.0465  False
    fc     rf  -0.0012    0.9 -0.0923 0.0899  False
    fc    svc  -0.0811  0.023 -0.1723   0.01  False
    rf    svc    -0.08 0.0262 -0.1711 0.0112  False
---------------------------------------------------


In [28]:
mc = MultiComparison(df60['f1score'], df60['methods'])
mc_results = mc.tukeyhsd(model60.f_pvalue)
print(mc_results)

Multiple Comparison of Means - Tukey HSD, FWER=0.07 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
    3c     dt  -0.0297 0.7794 -0.1004   0.041  False
    3c     fc   0.0199    0.9 -0.0508  0.0907  False
    3c     rf  -0.0102    0.9 -0.0809  0.0605  False
    3c    svc  -0.0539 0.2665 -0.1246  0.0168  False
    dt     fc   0.0496 0.3503 -0.0211  0.1204  False
    dt     rf   0.0195    0.9 -0.0512  0.0902  False
    dt    svc  -0.0242 0.8946 -0.0949  0.0465  False
    fc     rf  -0.0301 0.7701 -0.1009  0.0406  False
    fc    svc  -0.0739 0.0495 -0.1446 -0.0031   True
    rf    svc  -0.0437 0.4835 -0.1144   0.027  False
----------------------------------------------------
